In [10]:
import pandas as pd
import re
import operator

df = pd.read_csv(
    "./court_1_requiredOCR_False.csv", usecols=["판례내용", "file_name"]
)
pd.set_option("display.max_rows", 8000)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 20000)
print(df.shape)

with open("./crime_extract.txt", "r") as f:
    crime_ext = f.read()
crime_regex = re.compile(
    r"(\b사 *건\b)?(?(1)|(\b\d+[가-힣]{1,3}\d+\b))(?P<target>.*?)(\n *피 *고 *인\b)?(?(4)|(\b피 *고 *인\b))",
    re.DOTALL,
)

print('--')
def extract_crime_1(x):
    try:
        crime = crime_regex.search(x["판례내용"]).group("target")
        # crime = re.split(r'\n사 *건\b|\b사 *건 *번 *호\b|\n피 *고 *인\b', maxsplit=2, string=x['판례내용'])[1]
        # crime = re.search(r'(\b사 *건\b)?(?(1)|(\b\d+[가-힣]{1,3}\d+\b))(?P<target>.*?)(\n *피 *고 *인\b)?(?(4)|(\b피 *고 *인\b))', x['판례내용'], re.DOTALL).group('target')

    except AttributeError:
        return "x"
        # return x['판례내용']
    return crime


def extract_crime_2(x):
    text = x["사건-피고인"]
    text = re.sub(r"\n", "", text)
    crime = re.findall(rf"{crime_ext}", text)
    crime = list(set(crime))
    return crime


df["사건-피고인"] = df.apply(extract_crime_1, axis=1)
print('--')
df["test"] = df.apply(extract_crime_2, axis=1)
print(df.shape)
# df = df[list(map(operator.not_,[True if i != 'x' else False for i in df['사건-피고인']]))]
# df = df[[True if i != 'x' else False for i in df['사건-피고인']]]
df = df[[True if len(i) != 0 else False for i in df["test"]]]
print(df.shape)
# print(df[['사건-피고인',"file_name"]])
print(df[["test", "사건-피고인"]])
df.to_csv(
    "../csv_arrangement/test3.csv", index=False, columns=["test","사건-피고인"]
)


(6024, 2)
--
--
(6024, 4)
(3545, 4)
                                                   test                                             사건-피고인
1                                                [교통사고]                            2011고정1311  교통사고처리특례법위반
2                                                  [사기]                             2019고단363   사기, 변호사법위반
3                                                  [횡령]                                     2007고단5780  횡령
4                                                  [상해]                                     2020고단1142  상해
5                                                  [사기]                                      2008고정2440 사기
6                                                  [상해]                            2016고단3997   상해, 공무집행방해
7     [라. 도로교통법위반(무면허운전), 나. 도로교통법위반(사고후미조치), 가. 교통사...   2017고단387  가. 교통사고처리특례법위반(치상)\n             나...
8                                                [강도상해]                                    2014고합256  강도상해
9

In [12]:
import pandas as pd
import re
import operator

df = pd.read_csv(
    "./c_1_extract_case.csv", usecols=["판례내용", "file_name"]
)
pd.set_option("display.max_rows", 8000)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 20000)
print(df.shape)
pi2gumRegex = re.compile(r'(\n *피 *고 *인\b)(?P<target>.*?)(\n *검 *사\b)', re.DOTALL)


def extract_pi2gum(x): 
    text = x['판례내용']
    try: 
        result = pi2gumRegex.search(text).group('target')
        return result
    except AttributeError:
        print('===============')
        print(text[40:250])
        print('===============')

df['피고인-검사'] = df.apply(extract_pi2gum, axis=1)
print(df['피고인-검사'])

(3545, 2)
 ○ 사건번호  :  2006고단2222,  2007고단288(병합),  2007고단290(병합),  2007고단375(병
합)
  ○ 선고기일 : 2007. 6. 12.
  ○ 죄명 :  가. 폭력행위등처벌에관한법률위반(집단.흉기등상해)
나. 폭력행위등처벌에관한법률위반(집단.흉기등주거침입)
다. 폭력행위등처벌에
137 성폭법위반(특수강도강간등) 등 
 피   고   인  윤 ○○ (1968년생, 무직)
 판결선고일  2006. 1. 17. 
 강간 사실 인정 여부
 쟁      점
 성폭법위반(특수강도강간등)죄의 양형
 결과 (주문)  징역 10년
 법  정  형  사형․무기 또는 10년 이상의 징역
□ 판결 요지
 ○
사 부
판           결
사       건 2012고합399  성폭력범죄의처벌등에관한특례법위반(특수강도강간
등)
2012전고21(병합)  부착명령
피고인 겸 피부착명령청구자
배□■ 80년생, 남남남
  □ 사건의 경과
 사 건 번 호    2005고단986  모욕 등
 피  고  인    김○○
 공소제기일    2005. 6. 30.
 판결 선고일    2006. 1. 25.
 
 가. 형법상 모욕죄에서의 모욕의 의미
 쟁      점  나.  모욕행위가  형법  제20조의  정당행위로서  위법성  
  
한 소극적인 저항은 무죄」 
- 청주지방법원 2008. 5. 22. 선고 2007고정1442 판결 -
○ 사건의 개요 
사    건 : 2007고정1442  공무집행방해, 상해
공소제기 : 2007. 12. 7. 
피 고 인 : 김○○(1981년생 남자)
판결선고 : 2008. 5. 22
선고결과 : 무죄
☞ 이
사 부
판           결
사       건 2012고합373   폭력행위등처벌에관한법률위반(집단·흉기등존속협
박), 특정범죄가중처벌등에관한법률위반(보복범죄
등), 폭력행위등처벌에관한법률위반(상습협박), 존
속협박
피  고  인 문◇○63년생, 남
사 부
판           결
사       건 2